In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'What is the change in the genetic composition of a population over time?'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
#Will implement entity list in the future
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
#Will implement function to standardize proximity scores in the future
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [2]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '11',
 '_score': 8.522448,
 '_source': {'chapter': 5,
  'paragraph': 11,
  'content': 'Earth’s biodiversity is the product of evolution,  which can be defined as a change in the genetic composition of a population over time. Evolution can  occur at multiple levels. Evolution below the species  level, such as the evolution of different varieties of  apples or potatoes, is called microevolution. In contrast, when genetic changes give rise to new species, or  to new genera, families, classes, or phyla—larger categories of organisms into which species are organized—  we call the process macroevolution. Among these  many levels of macroevolution, the term speciation is  restricted to the evolution of new species.'}}

In [3]:
s = 'The number of species in a given area is known as what?'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
#Will implement entity list in the future
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
#Will implement function to standardize proximity scores in the future
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [4]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '6',
 '_score': 4.1437445,
 '_source': {'chapter': 5,
  'paragraph': 6,
  'content': 'The number of species in a given area, such as a  pond, the canopy of a tree, or a plot of grassland, is  known as species richness. Species richness is used  to give an approximate sense of the biodiversity of a  particular place. However, we may also want to know  the species evenness, which is the relative proportion of individuals within the different species in a  location. Species evenness tells us whether a particular  ecosystem is numerically dominated by one species or  whether all of its species have similar abundances. An  ecosystem has high species evenness if its species are  all represented by similar numbers of individuals. An  ecosystem has low species evenness if one species is  represented by many individuals whereas other species are represented by only a few individuals. In this  case, there is effectively less diversity.'}}

In [5]:
s = 'This is a result of the phenomenon resulting in catastrophic failure.'
doc = nlp(s)
[(ent.text, ent.label_) for ent in doc.ents]

[]

In [6]:
doc

This is a result of the phenomenon resulting in catastrophic failure.

In [7]:
[token.text for token in doc]

['This',
 'is',
 'a',
 'result',
 'of',
 'the',
 'phenomenon',
 'resulting',
 'in',
 'catastrophic',
 'failure',
 '.']

In [8]:
[token.ent_type for token in doc]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
[token.pos_ for token in doc]

['DET',
 'VERB',
 'DET',
 'NOUN',
 'ADP',
 'DET',
 'NOUN',
 'VERB',
 'ADP',
 'ADJ',
 'NOUN',
 'PUNCT']

In [10]:
[(token.text, token.pos_) for token in doc]

[('This', 'DET'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('result', 'NOUN'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('phenomenon', 'NOUN'),
 ('resulting', 'VERB'),
 ('in', 'ADP'),
 ('catastrophic', 'ADJ'),
 ('failure', 'NOUN'),
 ('.', 'PUNCT')]

In [11]:
s = 'Phenomena resulting in catastrophic failure.'

In [13]:
[(token.text, token.pos_, token.lemma_) for token in doc]

[('This', 'DET', 'this'),
 ('is', 'VERB', 'be'),
 ('a', 'DET', 'a'),
 ('result', 'NOUN', 'result'),
 ('of', 'ADP', 'of'),
 ('the', 'DET', 'the'),
 ('phenomenon', 'NOUN', 'phenomenon'),
 ('resulting', 'VERB', 'result'),
 ('in', 'ADP', 'in'),
 ('catastrophic', 'ADJ', 'catastrophic'),
 ('failure', 'NOUN', 'failure'),
 ('.', 'PUNCT', '.')]

In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'What is the change in the genetic composition of a population over time?'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
#Will implement entity list in the future
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
#Will implement function to standardize proximity scores in the future
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [2]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '11',
 '_score': 8.522448,
 '_source': {'chapter': 5,
  'paragraph': 11,
  'content': 'Earth’s biodiversity is the product of evolution,  which can be defined as a change in the genetic composition of a population over time. Evolution can  occur at multiple levels. Evolution below the species  level, such as the evolution of different varieties of  apples or potatoes, is called microevolution. In contrast, when genetic changes give rise to new species, or  to new genera, families, classes, or phyla—larger categories of organisms into which species are organized—  we call the process macroevolution. Among these  many levels of macroevolution, the term speciation is  restricted to the evolution of new species.'}}

In [5]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'Artificial selection proceeds at a much quicker rate than natural selection'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
#Will implement entity list in the future
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
#Will implement function to standardize proximity scores in the future
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [6]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '30',
 '_score': 7.964067,
 '_source': {'chapter': 5,
  'paragraph': 30,
  'content': 'Artificial and natural selection are important mechanisms of evolution, but evolution can also occur by  random, or nonadaptive, processes. In these cases, the  genetic composition of a population changes over  time, but the changes are not related to differences in  fitness among individuals. There are five random processes: mutation, gene flow, genetic drift, bottleneck effects,  and founder effects.'}}

In [7]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'Artificial selection natural selection quicker'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
#Will implement entity list in the future
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
#Will implement function to standardize proximity scores in the future
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [8]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '30',
 '_score': 7.964067,
 '_source': {'chapter': 5,
  'paragraph': 30,
  'content': 'Artificial and natural selection are important mechanisms of evolution, but evolution can also occur by  random, or nonadaptive, processes. In these cases, the  genetic composition of a population changes over  time, but the changes are not related to differences in  fitness among individuals. There are five random processes: mutation, gene flow, genetic drift, bottleneck effects,  and founder effects.'}}

In [9]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'Artificial selection quicker'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
#Will implement entity list in the future
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
#Will implement function to standardize proximity scores in the future
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [10]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '21',
 '_score': 5.0261393,
 '_source': {'chapter': 5,
  'paragraph': 21,
  'content': 'When humans determine which individuals to  breed, typically with a preconceived set of traits in  mind, we call the process evolution by artificial  selection. Artificial selection has produced numerous  breeds of horses, cattle, sheep, pigs, and chickens with  traits that humans find useful or aesthetically pleasing.  Most of our modern agricultural crops are also the  result of many years of careful breeding. For example,  starting with a single species of wild mustard, Brassica  oleracea, plant breeders have produced a variety of food crops, including cabbage, cauliflower, broccoli, Brussels  sprouts, kale, and kohlrabi.'}}

In [11]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'adaptaion mutation'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
#Will implement entity list in the future
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
#Will implement function to standardize proximity scores in the future
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x1217130f0>: Failed to establish a new connection: [Errno 61] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x1217130f0>: Failed to establish a new connection: [Errno 61] Connection refused)

In [12]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'The number of species in a given area is known as'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
#Will implement entity list in the future
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
#Will implement function to standardize proximity scores in the future
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [13]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '6',
 '_score': 4.1437445,
 '_source': {'chapter': 5,
  'paragraph': 6,
  'content': 'The number of species in a given area, such as a  pond, the canopy of a tree, or a plot of grassland, is  known as species richness. Species richness is used  to give an approximate sense of the biodiversity of a  particular place. However, we may also want to know  the species evenness, which is the relative proportion of individuals within the different species in a  location. Species evenness tells us whether a particular  ecosystem is numerically dominated by one species or  whether all of its species have similar abundances. An  ecosystem has high species evenness if its species are  all represented by similar numbers of individuals. An  ecosystem has low species evenness if one species is  represented by many individuals whereas other species are represented by only a few individuals. In this  case, there is effectively less diversity.'}}

In [14]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'The range of abiotic and biotic conditions under which a species actually lives is called its '
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [15]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '52',
 '_score': 13.8511095,
 '_source': {'chapter': 5,
  'paragraph': 52,
  'content': 'The fundamental niche establishes the abiotic  limits for the persistence of a species. However,  biotic factors can further limit the locations where a  species can live. Common biotic limitations include  the presence of competitors, predators, and diseases.  For example, even if abiotic conditions are favorable  for a plant species in a particular location, other  plant species may be better competitors for water  and soil nutrients. Those competitors might prevent  the species from growing in that environment.  Similarly, even if a small rodent can tolerate the  temperature and humidity of a tropical forest, a  deadly rodent disease might prevent the species  from persisting in the forest. Therefore, biotic factors further narrow the fundamental niche that a  species actually uses. The range of abiotic and biotic  conditions under which a species

In [16]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'When humans determine which individuals breed, we call this'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [17]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '21',
 '_score': 4.2195663,
 '_source': {'chapter': 5,
  'paragraph': 21,
  'content': 'When humans determine which individuals to  breed, typically with a preconceived set of traits in  mind, we call the process evolution by artificial  selection. Artificial selection has produced numerous  breeds of horses, cattle, sheep, pigs, and chickens with  traits that humans find useful or aesthetically pleasing.  Most of our modern agricultural crops are also the  result of many years of careful breeding. For example,  starting with a single species of wild mustard, Brassica  oleracea, plant breeders have produced a variety of food crops, including cabbage, cauliflower, broccoli, Brussels  sprouts, kale, and kohlrabi.'}}

In [18]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'When humans determine which individuals breed, we call this'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [19]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '21',
 '_score': 4.2195663,
 '_source': {'chapter': 5,
  'paragraph': 21,
  'content': 'When humans determine which individuals to  breed, typically with a preconceived set of traits in  mind, we call the process evolution by artificial  selection. Artificial selection has produced numerous  breeds of horses, cattle, sheep, pigs, and chickens with  traits that humans find useful or aesthetically pleasing.  Most of our modern agricultural crops are also the  result of many years of careful breeding. For example,  starting with a single species of wild mustard, Brassica  oleracea, plant breeders have produced a variety of food crops, including cabbage, cauliflower, broccoli, Brussels  sprouts, kale, and kohlrabi.'}}

In [20]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'When humans determine which individuals breed, we call this'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [21]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '21',
 '_score': 4.2195663,
 '_source': {'chapter': 5,
  'paragraph': 21,
  'content': 'When humans determine which individuals to  breed, typically with a preconceived set of traits in  mind, we call the process evolution by artificial  selection. Artificial selection has produced numerous  breeds of horses, cattle, sheep, pigs, and chickens with  traits that humans find useful or aesthetically pleasing.  Most of our modern agricultural crops are also the  result of many years of careful breeding. For example,  starting with a single species of wild mustard, Brassica  oleracea, plant breeders have produced a variety of food crops, including cabbage, cauliflower, broccoli, Brussels  sprouts, kale, and kohlrabi.'}}

In [22]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'When humans determine which individuals breed, we call this'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [23]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

import spacy
nlp = spacy.load('en_core_web_sm')

s = 'When humans determine which individuals breed, we call this'
question = nlp(s)

search_ents = []
for token in question:
    if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
        search_ents.append(token)
        
search_ents_txt = [token.text for token in search_ents]
content = ' '.join(search_ents_txt)
        
res = es.search(index = 'chapter5', body = {
    'query': {
        'bool': {
            'must': [{
                'match': {
                    'content': content
                }
            }]
        }
    }
})

doc_hits = res['hits']['hits']
pars = [nlp(doc['_source']['content']) for doc in res['hits']['hits']]

def minimum_dist(a, b):
    if len(a) == 0 or len(b) == 0:
        return 100
    a_idx = 0
    b_idx = 0
    dist = abs(a[0] - b[0])
    while (a_idx < len(a) and b_idx < len(b)):
        if abs(a[a_idx] - b[b_idx]) < dist:
            dist = abs(a[a_idx] - b[b_idx])
        if a[a_idx] < b[b_idx]:
            a_idx += 1
        else:
            b_idx += 1
    return dist

def prox_sc(ent_pos_list):
    dist_sum = 0
    for i in range(len(ent_pos_list) - 1):
        for k in range(i + 1, len(ent_pos_list)):
            dist_sum += minimum_dist(ent_pos_list[i], ent_pos_list[k])
    dist_avg = float(dist_sum) / float(len(ent_pos_list))
    return dist_avg

prox_scores = []
for par in pars:
    ent_positions = []
    for i in range(len(search_ents)):
        ent_positions.append([token.i for token in par if token.text == search_ents_txt[i]])
    prox_scores.append(prox_sc(ent_positions))
    
best_hit = prox_scores.index(min(prox_scores))
best_par = doc_hits[best_hit]

In [24]:
best_par

{'_index': 'chapter5',
 '_type': '_doc',
 '_id': '21',
 '_score': 4.2195663,
 '_source': {'chapter': 5,
  'paragraph': 21,
  'content': 'When humans determine which individuals to  breed, typically with a preconceived set of traits in  mind, we call the process evolution by artificial  selection. Artificial selection has produced numerous  breeds of horses, cattle, sheep, pigs, and chickens with  traits that humans find useful or aesthetically pleasing.  Most of our modern agricultural crops are also the  result of many years of careful breeding. For example,  starting with a single species of wild mustard, Brassica  oleracea, plant breeders have produced a variety of food crops, including cabbage, cauliflower, broccoli, Brussels  sprouts, kale, and kohlrabi.'}}